In [3]:
import pyspark
from pyspark.sql import SparkSession

spark = SparkSession.builder \
    .master("local[*]") \
    .appName('hw-5') \
    .getOrCreate()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


23/03/03 22:04:11 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [4]:
spark.version

'3.3.2'

In [12]:
data_file = 'fhvhv_tripdata_2021-06.csv.gz'

In [112]:
%env URL=https://github.com/DataTalksClub/nyc-tlc-data/releases/download

env: URL=https://github.com/DataTalksClub/nyc-tlc-data/releases/download


In [113]:
!echo $$URL/fhvhv/{data_file}

https://github.com/DataTalksClub/nyc-tlc-data/releases/download/fhvhv/fhvhv_tripdata_2021-06.csv.gz


In [39]:
!pwd
!mkdir data
!ls -lH

/Users/duy.ton/CodeSpace/workshop/dezc_230/wk_5_batch_processing
mkdir: data: File exists
total 8
drwxr-xr-x  2 duy.ton  staff    64 Mar  3 22:55 data
-rw-r--r--  1 duy.ton  staff  3312 Mar  3 22:55 hw.ipynb


In [40]:
!curl -L $$URL/fhvhv/{data_file} -o data/{data_file}

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
100  167M  100  167M    0     0   917k      0  0:03:07  0:03:07 --:--:-- 1030k:03:18 1097k10  167M   10 16.8M    0     0   840k      0  0:03:24  0:00:20  0:03:04  720k  0  1003k      0  0:02:51  0:00:36  0:02:15 1189k0   974k      0  0:02:56  0:00:41  0:02:15  762k  0  0:03:00  0:01:15  0:01:45 1130kk      0  0:03:01  0:01:24  0:01:37 1050k 0   945k      0  0:03:01  0:01:47  0:01:14  604k  0  0:03:03  0:01:59  0:01:04  835k  0     0   921k      0  0:03:06  0:02:25  0:00:41  772k


In [43]:
!ls -lH data

total 360456
-rw-r--r--  1 duy.ton  staff  175799316 Mar  3 23:00 fhvhv_tripdata_2021-06.csv.gz


In [44]:
import pyspark.sql.types as Tp
import pyspark.sql.functions as Fn

In [56]:
fhvhv_schema = Tp.StructType([
    Tp.StructField('dispatching_base_num', Tp.StringType(), True),
    Tp.StructField('pickup_datetime', Tp.TimestampType(), True),
    Tp.StructField('dropoff_datetime', Tp.TimestampType(), True),
    Tp.StructField('PULocationID', Tp.IntegerType(), True),
    Tp.StructField('DOLocationID', Tp.IntegerType(), True),
    Tp.StructField('SR_Flag', Tp.StringType(), True),
    Tp.StructField('affiliated_base_number', Tp.StringType(), True),
])

In [57]:
df = spark.read.schema(fhvhv_schema).csv(f'data/{data_file}', header=True)

In [61]:
df.printSchema()

root
 |-- dispatching_base_num: string (nullable = true)
 |-- pickup_datetime: timestamp (nullable = true)
 |-- dropoff_datetime: timestamp (nullable = true)
 |-- PULocationID: integer (nullable = true)
 |-- DOLocationID: integer (nullable = true)
 |-- SR_Flag: string (nullable = true)
 |-- affiliated_base_number: string (nullable = true)



In [58]:
df.show(5)

+--------------------+-------------------+-------------------+------------+------------+-------+----------------------+
|dispatching_base_num|    pickup_datetime|   dropoff_datetime|PULocationID|DOLocationID|SR_Flag|affiliated_base_number|
+--------------------+-------------------+-------------------+------------+------------+-------+----------------------+
|              B02764|2021-06-01 00:02:41|2021-06-01 00:07:46|         174|          18|      N|                B02764|
|              B02764|2021-06-01 00:16:16|2021-06-01 00:21:14|          32|         254|      N|                B02764|
|              B02764|2021-06-01 00:27:01|2021-06-01 00:42:11|         240|         127|      N|                B02764|
|              B02764|2021-06-01 00:46:08|2021-06-01 00:53:45|         127|         235|      N|                B02764|
|              B02510|2021-06-01 00:45:42|2021-06-01 01:03:33|         144|         146|      N|                  null|
+--------------------+------------------

In [62]:
df_partitioned = df.repartition(12)
df_partitioned.write.parquet(f'data/12-partitions/', mode='overwrite')

23/03/03 23:10:31 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 95.00% for 8 writers


In [65]:
!ls -lh data/12-partitions

total 591360
-rw-r--r--  1 duy.ton  staff     0B Mar  3 23:10 _SUCCESS
-rw-r--r--  1 duy.ton  staff    24M Mar  3 23:10 part-00000-d6aad2ba-7152-48d6-9515-3d3bf6a6ae1b-c000.snappy.parquet
-rw-r--r--  1 duy.ton  staff    24M Mar  3 23:10 part-00001-d6aad2ba-7152-48d6-9515-3d3bf6a6ae1b-c000.snappy.parquet
-rw-r--r--  1 duy.ton  staff    24M Mar  3 23:10 part-00002-d6aad2ba-7152-48d6-9515-3d3bf6a6ae1b-c000.snappy.parquet
-rw-r--r--  1 duy.ton  staff    24M Mar  3 23:10 part-00003-d6aad2ba-7152-48d6-9515-3d3bf6a6ae1b-c000.snappy.parquet
-rw-r--r--  1 duy.ton  staff    24M Mar  3 23:10 part-00004-d6aad2ba-7152-48d6-9515-3d3bf6a6ae1b-c000.snappy.parquet
-rw-r--r--  1 duy.ton  staff    24M Mar  3 23:10 part-00005-d6aad2ba-7152-48d6-9515-3d3bf6a6ae1b-c000.snappy.parquet
-rw-r--r--  1 duy.ton  staff    24M Mar  3 23:10 part-00006-d6aad2ba-7152-48d6-9515-3d3bf6a6ae1b-c000.snappy.parquet
-rw-r--r--  1 duy.ton  staff    24M Mar  3 23:10 part-00007-d6aad2ba-7152-48d6-9515-3d3bf6a6ae1b-c000.snappy.p

In [72]:
# df_jun_15 = df.filter(Fn.to_date(df.pickup_datetime) == '2021-06-15')
# a more elaborated version of above
col_pickup_dt = Fn.col('pickup_datetime')
df_jun_15 = df \
    .filter(Fn.month(col_pickup_dt) == 6) \
    .filter(Fn.dayofmonth(col_pickup_dt) == 15)

df_jun_15.count()

452470

In [74]:
df.dtypes

[('dispatching_base_num', 'string'),
 ('pickup_datetime', 'timestamp'),
 ('dropoff_datetime', 'timestamp'),
 ('PULocationID', 'int'),
 ('DOLocationID', 'int'),
 ('SR_Flag', 'string'),
 ('affiliated_base_number', 'string')]

In [111]:
calc_trip_duration = df.dropoff_datetime - df.pickup_datetime

df_max_trip = df \
    .withColumn('trip_duration', calc_trip_duration) \
    .select(Fn.max('trip_duration').alias('longest'))

df_max_trip \
    .withColumn('longest_secs', Fn.col('longest').cast('long')) \
    .withColumn('longest_hrs', Fn.round(Fn.col('longest_secs') / 3600, 2)) \
    .show(truncate=False)

+-----------------------------------+------------+-----------+
|longest                            |longest_secs|longest_hrs|
+-----------------------------------+------------+-----------+
|INTERVAL '2 18:52:44' DAY TO SECOND|240764      |66.88      |
+-----------------------------------+------------+-----------+



In [120]:
data_zones = 'taxi_zone_lookup.csv'

In [126]:
!curl -L $$URL/misc/{data_zones} -o data/{data_zones}

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
100 12322  100 12322    0     0      0      0 --:--:-- --:--:-- --:--:--     00     0  10366      0  0:00:01  0:00:01 --:--:-- 31757


In [128]:
df_zones = spark.read.csv(f'data/{data_zones}', header=True)

In [130]:
df_zones.show(5)

+----------+-------------+--------------------+------------+
|LocationID|      Borough|                Zone|service_zone|
+----------+-------------+--------------------+------------+
|         1|          EWR|      Newark Airport|         EWR|
|         2|       Queens|         Jamaica Bay|   Boro Zone|
|         3|        Bronx|Allerton/Pelham G...|   Boro Zone|
|         4|    Manhattan|       Alphabet City| Yellow Zone|
|         5|Staten Island|       Arden Heights|   Boro Zone|
+----------+-------------+--------------------+------------+
only showing top 5 rows



In [132]:
df_joined = df.join(df_zones, on=[df.PULocationID == df_zones.LocationID])

In [136]:
df_joined \
    .withColumnRenamed('Zone', 'pickup_zone') \
    .groupBy('pickup_zone').count() \
    .orderBy('count', ascending=False) \
    .limit(10) \
    .show(truncate=False)

+--------------------+------+
|pickup_zone         |count |
+--------------------+------+
|Crown Heights North |231279|
|East Village        |221244|
|JFK Airport         |188867|
|Bushwick South      |187929|
|East New York       |186780|
|TriBeCa/Civic Center|164344|
|LaGuardia Airport   |161596|
|Union Sq            |158937|
|West Village        |154698|
|Astoria             |152493|
+--------------------+------+

